## Load Data

In [1]:
!ls

LICENSE
Mod4_Project_Tino.ipynb
Mod4_Project_Tino_MaryJo.ipynb
README.md
column_names.md
kc_housing_data_for_feat_engineering_lab.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 300)

In [3]:
df_orig = pd.read_csv("kc_housing_data_for_feat_engineering_lab.csv")

In [4]:
df_orig.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,yr_old,year_sold,since_sold,price_log
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,62,2014,3,12.309982
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,66,2014,3,13.195614
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,84,2015,2,12.100712
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,52,2014,3,13.311329
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,30,2015,2,13.142166


## Feature Engineering

In [5]:
df = df_orig.copy()

### Creating a Price per sqft by zipcode Feature

In [6]:
# Calculated field created for price/sqft
df['price_sqft'] = df['price']/ df['sqft_living']

In [7]:
grouped = df.groupby('zipcode')
grouped_psqft = grouped['price_sqft'].agg(np.mean)

#aggregate
df = df.merge(grouped_psqft, left_on=['zipcode'], right_on=['zipcode'])
df.rename(columns={'price_sqft_y':'price_sqft_mean_zip'}, inplace=True)

### Distance to biggest Employers
We assume Hoosing prices are effected by the distance to the biggest employers in the area. The three biggest Employers are:
* Boing
* Microsoft
* Amazon <br>
therefore three Columns are created estimating the distance with log lang data

In [8]:
#Lat and Long Data of the Employers
employers_dict = { "boeing": {"lat":47.6213723 , "long":-122.2890233 },
                  "microsoft":{"lat":47.6423318 , "long":-122.1456849},
                  "amazon":{"lat":47.6222917 , "long":-122.3386826 }}
                     

In [9]:
# Distance estimation using geometric distance
def get_distance(lat,long,t_lat,t_long):
    dx = lat-t_lat
    dy = long-t_long
    return (dx**2+dy**2)**0.5


def get_emp_distance(lat,long,employer):
    t_lat = employers_dict[employer]["lat"]
    t_long = employers_dict[employer]["long"]
    d = get_distance(lat,long,t_lat,t_long)
    return d

In [10]:
# Create Distance Features for Boing, Amazon , Microsoft
for emp in employers_dict.keys():
    print(emp)
    df[emp] = [get_emp_distance(df.iloc[i]["lat"],df.iloc[i]["long"],emp) for i in range(len(df))]

boeing
microsoft
amazon


## Normalization

In [11]:
df["zipcode"] = df.zipcode.astype("str")

In [24]:
df["yr_renovated"] = df["yr_renovated"].apply(lambda x: 1 if x>0 else 0)
df = df.rename(columns = {"yr_renovated":"renovated"})

In [25]:
df_with_features = df.copy()

In [30]:
df_with_features.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,renovated,zipcode,lat,long,sqft_living15,sqft_lot15,yr_old,year_sold,since_sold,price_log,price_sqft_x,price_sqft_mean_zip,boeing,microsoft,amazon
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,62,2014,3,12.309982,188.050847,189.172528,0.114732,0.172008,0.137889
1,4060000240,2014-06-23,205425.0,2,1.00,880,6780,1.0,0,0,4,6,880,0,1945,0,98178,47.5009,-122.248,1190,6780,72,2014,3,12.232836,233.437500,189.172528,0.127265,0.174560,0.151523
2,4058801670,2014-07-17,445000.0,3,2.25,2100,8201,1.0,0,2,3,8,1620,480,1967,0,98178,47.5091,-122.244,2660,8712,50,2014,3,13.005830,211.904762,189.172528,0.120963,0.165580,0.147571
3,2976800796,2014-09-25,236000.0,3,1.00,1300,5898,1.0,0,0,3,7,1300,0,1961,0,98178,47.5053,-122.255,1320,7619,56,2014,3,12.371587,181.538462,189.172528,0.120956,0.175293,0.143840
4,6874200960,2015-02-27,170000.0,2,1.00,860,5265,1.0,0,0,3,6,860,0,1931,0,98178,47.5048,-122.272,1650,8775,86,2015,2,12.043554,197.674419,189.172528,0.117809,0.186736,0.135096


In [34]:
# Train-Test-Split
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import
X = df.drop(["price", "price_log", "id" ,"date","year_sold", "price_sqft_x"], axis = 1)
y = df.price_log
X_train, y_train , X_test, y_test = train_test_split(X,y, random_state = 34)

In [35]:
X_train.head()


,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,renovated,zipcode,lat,long,sqft_living15,sqft_lot15,yr_old,since_sold,price_sqft_mean_zip,boeing,microsoft,amazon
20784,3,2.75,2550,54014,2.0,0,0,4,8,1980,570,1967,0,98072,47.7596,-122.117,2180,21600,50,3,247.512013,0.220678,0.120726,0.260762
8093,2,1.50,1760,12000,1.0,0,0,4,7,1760,0,1964,0,98052,47.6288,-122.109,2200,12088,53,3,280.386324,0.180176,0.039101,0.229775
2521,3,3.00,2790,12523,2.0,1,4,4,8,1600,1190,1977,0,98198,47.3571,-122.324,2990,11476,40,3,178.428610,0.266577,0.336383,0.265598
18569,5,4.25,3920,11412,2.0,0,0,3,7,3920,0,1955,1,98006,47.5766,-122.151,1400,9750,62,2,298.555259,0.145103,0.065946,0.193164
11177,5,3.50,2738,6031,2.0,0,0,3,8,2738,0,2014,0,98023,47.2962,-122.350,2738,5201,3,3,148.919829,0.330840,0.401935,0.326288
